In [354]:
import pandas as pd
import numpy as np
import cPickle 
from nltk.corpus import stopwords
from gensim.models import word2vec
import nltk.data
import re
import logging
from nltk.stem.snowball import *
import itertools

# Python 2.x:
import HTMLParser
html_parser = HTMLParser.HTMLParser()

import multiprocessing
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)   

from nltk.stem import WordNetLemmatizer, PorterStemmer
wordnet_lemmatizer = WordNetLemmatizer()
porter = PorterStemmer()

In [355]:
TEST_CASE = {}

In [394]:
import re
import HTMLParser
html_parser = HTMLParser.HTMLParser()
from nltk.stem import WordNetLemmatizer, PorterStemmer
wordnet_lemmatizer = WordNetLemmatizer()
porter = PorterStemmer()

def skill_transform(skill, remove_stopwords = True):
    skill = unicode(skill)
    skill = html_parser.unescape(skill)
    
    skill = skill.replace("_", " ").split()
    skill = " ".join([sk for sk in skill if sk])
    
    skill = re.sub(r"\(.*\)", "", skill)
    skill = skill.replace("-", "") \
        .replace(".", "") \
        .replace(",", "") \
        .replace("-", "") \
        .replace(":", "") \
        .replace("(", "") \
        .replace(")", "") \
        .replace(u"åá", "") \
        .replace(u"&", "and") \
        .replace(" js", "js") \
        .replace("-js", "js") \
        .replace("_js", "js") \
        .replace("java script", "js") 
    
    skill = skill.lower()
    
    # Special cases replace
    special_case = {}
    special_case["javascript"] = [ "js", "java script", "javascripts", "java scrip" ]
    special_case["wireframe"] = [ "wireframes", "wire frame", "wire frames", "wire-frame", "wirefram", "wire fram", "wireframing" ]
    special_case["OOP"] = [  "object oriented", "object oriented programming", ]
    special_case["OOD"] = [ "object oriented design", ]
    special_case["OLAP"] = [ "online analytical processing",  ]
    special_case["Ecommerce"] = [ "e commerce",  ]
    special_case["consultant"] = [ "consulting",  ]
    special_case["ux"] = [ "user experience", "web user experience design", "user experience design", "ux designer", "user experience/ux" ]
    special_case["html5"] = [ "html 5",  ]
    special_case["j2ee"] = [ "jee",  ]
    special_case["osx"] = [ "mac os x", "os x" ]
    special_case["senior"] = [ "sr" ]
    special_case["qa"] = [ "quality",  ]
    special_case["bigdata"] = [ "big data",  ]
    special_case["webservice"] = [ "webservices", "website", "webapps" ]
    special_case["xml"] = [ "xml file", "xml schemas", "xml/json", "xml web service" ]
    special_case["bigdata"] = [ "big data",  ]
    special_case["nlp"] = [ "natural language process", "natural language", "nltk" ]
    for root_skill in special_case:
        if skill in special_case[root_skill]:
            skill = root_skill
    
    # Special case regex
    special_case_regex = {
        r'^angular.*$': 'angularjs',
        r'^node.*$': 'nodejs',
        r'^(.*)[_\s]js$': '\\1js',
        r'^(.*) js$': '\\1js',
    }
    for regex_rule in special_case_regex:
        after_skill = re.sub(regex_rule, special_case_regex[regex_rule], skill)
        if after_skill != skill:
            skill = after_skill
            break
    
    # Stem
    if len(skill) > 2:
        skill_after = skill.split(" ")
        skill_after = [wordnet_lemmatizer.lemmatize(sk, pos="v") for sk in skill_after]
        skill_after = " ".join(skill_after)
        skill = skill_after
    
    # skill stopwords 
    if remove_stopwords:
        skill_stopwords = [ "app", "touch", "the", "application" ]
        skill_after = skill.split(" ")
        skill = " ".join([ sk for sk in skill_after if sk not in skill_stopwords ])
    
    skill = skill.lower().strip().replace(" ", "_")
    skill = re.sub(' +',' ', skill)
    
    # NOTE: replace js tail
    skill = re.sub('js$','', skill)
    
    return skill

skill_transform("js")
skill_transform("angularjs")

u'angular'

In [357]:
def test_case(model, model_name):
    cases = [ "machine learning", "js", "javascript", "python", "html", "html5", "css", "angular", "nodejs" ]
    TEST_CASE[model_name] = []
    for case in cases:
        try:
            TEST_CASE[model_name].append(model.similar_by_word(skill_transform(case)))
        except:
            pass

In [358]:
data_must_have = pd.read_csv('mustHaveSkills-2.csv', header = 0, encoding='ISO-8859-1')
data_must_have.drop_duplicates(inplace=True, keep='last')
data_must_have = data_must_have[data_must_have["job_title"] != 0]
del data_must_have["job_brief_id"]
# data_must_have['Count'] = data_must_have.groupby('job_title')['keyword_name'].transform(pd.Series.value_counts)
data_must_have.drop_duplicates(inplace=True)

unique_skill_before = len(data_must_have['keyword_name'].unique())
print "Before: %s" % unique_skill_before

data_must_have['skill'] = data_must_have['keyword_name'].apply(skill_transform)

unique_skill_after = len(data_must_have['skill'].unique())
print "After: %s" % unique_skill_after 

print "Resize: %s%%" % (unique_skill_after * 100.0 / unique_skill_before)

# data_must_have['keyword_name'] = data_must_have['keyword_name'].str.replace(' ' ,'_')
# data_must_have

Before: 8589
After: 6341
Resize: 73.8269880079%


In [359]:
# pd.set_option('display.max_rows', 1500)
# data_must_have.drop_duplicates(subset=["keyword_name", "skill"])[["keyword_name", "skill"]].sort_values("skill")[3000:]
musthave_data = data_must_have.groupby('job_title')['skill'].apply(list).tolist()

print ("Training model...")
musthave_model = word2vec.Word2Vec(musthave_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
          size=300, 
          min_count=1, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
          iter=4,
          sg =1
    )

musthave_model.init_sims(replace=True)
test_case(musthave_model, "musthave_model")

2017-04-17 13:02:44,320 : INFO : collecting all words and their counts
2017-04-17 13:02:44,322 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-04-17 13:02:44,366 : INFO : collected 6341 word types from a corpus of 80810 raw words and 5649 sentences
2017-04-17 13:02:44,367 : INFO : Loading a fresh vocabulary
2017-04-17 13:02:44,396 : INFO : min_count=1 retains 6341 unique words (100% of original 6341, drops 0)
2017-04-17 13:02:44,397 : INFO : min_count=1 leaves 80810 word corpus (100% of original 80810, drops 0)
2017-04-17 13:02:44,419 : INFO : deleting the raw counts dictionary of 6341 items
2017-04-17 13:02:44,421 : INFO : sample=0.001 downsamples 34 most-common words
2017-04-17 13:02:44,422 : INFO : downsampling leaves estimated 77635 word corpus (96.1% of prior 80810)
2017-04-17 13:02:44,423 : INFO : estimated required memory for 6341 words and 300 dimensions: 18388900 bytes
2017-04-17 13:02:44,444 : INFO : resetting layer weights


Training model...


2017-04-17 13:02:44,579 : INFO : training model with 32 workers on 6341 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=10
2017-04-17 13:02:44,845 : INFO : worker thread finished; awaiting finish of 31 more threads
2017-04-17 13:02:44,894 : INFO : worker thread finished; awaiting finish of 30 more threads
2017-04-17 13:02:44,928 : INFO : worker thread finished; awaiting finish of 29 more threads
2017-04-17 13:02:44,979 : INFO : worker thread finished; awaiting finish of 28 more threads
2017-04-17 13:02:45,053 : INFO : worker thread finished; awaiting finish of 27 more threads
2017-04-17 13:02:45,423 : INFO : worker thread finished; awaiting finish of 26 more threads
2017-04-17 13:02:45,448 : INFO : worker thread finished; awaiting finish of 25 more threads
2017-04-17 13:02:45,462 : INFO : worker thread finished; awaiting finish of 24 more threads
2017-04-17 13:02:45,545 : INFO : worker thread finished; awaiting finish of 23 more threads
2017-04-17 13:02:45,7

In [360]:
with open("prep_data_tokens_underscore_1", "rb") as g:
    data_dice = cPickle.load(g)
dice_data = [ [ skill_transform(d.replace("_", " ")) for d in dice if d is not None ] for dice in data_dice if dice is not None ]


musthave_dice_data = dice_data + musthave_data
musthave_dice_data = [ list(set([ d for d in data if d is not None and len(d) > 0 ])) for data in musthave_dice_data  ]

print len(musthave_dice_data)
musthave_dice_data

print ("Training model...")
musthave_dice_model = word2vec.Word2Vec(musthave_dice_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
          size=300, 
          min_count=1, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
          iter=4,
          sg =1
    )

musthave_dice_model.init_sims(replace=True)
test_case(musthave_dice_model, "musthave_dice_model")

2017-04-17 13:02:55,676 : INFO : collecting all words and their counts
2017-04-17 13:02:55,678 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-04-17 13:02:55,759 : INFO : PROGRESS: at sentence #10000, processed 217169 words, keeping 11788 word types
2017-04-17 13:02:55,774 : INFO : collected 12749 word types from a corpus of 255598 raw words and 12795 sentences
2017-04-17 13:02:55,775 : INFO : Loading a fresh vocabulary
2017-04-17 13:02:55,812 : INFO : min_count=1 retains 12749 unique words (100% of original 12749, drops 0)
2017-04-17 13:02:55,814 : INFO : min_count=1 leaves 255598 word corpus (100% of original 255598, drops 0)
2017-04-17 13:02:55,843 : INFO : deleting the raw counts dictionary of 12749 items
2017-04-17 13:02:55,844 : INFO : sample=0.001 downsamples 1 most-common words
2017-04-17 13:02:55,845 : INFO : downsampling leaves estimated 255585 word corpus (100.0% of prior 255598)
2017-04-17 13:02:55,846 : INFO : estimated required memory for 1

12795
Training model...


2017-04-17 13:02:55,881 : INFO : resetting layer weights
2017-04-17 13:02:56,071 : INFO : training model with 32 workers on 12749 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=10
2017-04-17 13:02:57,157 : INFO : PROGRESS: at 11.99% examples, 139878 words/s, in_qsize 64, out_qsize 0
2017-04-17 13:02:58,172 : INFO : PROGRESS: at 46.24% examples, 225021 words/s, in_qsize 56, out_qsize 0
2017-04-17 13:02:59,197 : INFO : PROGRESS: at 68.49% examples, 232152 words/s, in_qsize 28, out_qsize 9
2017-04-17 13:02:59,238 : INFO : worker thread finished; awaiting finish of 31 more threads
2017-04-17 13:02:59,263 : INFO : worker thread finished; awaiting finish of 30 more threads
2017-04-17 13:02:59,266 : INFO : worker thread finished; awaiting finish of 29 more threads
2017-04-17 13:02:59,281 : INFO : worker thread finished; awaiting finish of 28 more threads
2017-04-17 13:02:59,283 : INFO : worker thread finished; awaiting finish of 27 more threads
2017-04-17 13:02:59

In [361]:
# model.similar_by_word(skill_transform('machine learning'))

In [362]:
# model.similar_by_word('python')

In [363]:
musthave_dice_naruki_data = musthave_dice_data
data_naruki = pd.read_csv('naukri_skill_full', header = 0, encoding='ISO-8859-1')
data_naruki.drop_duplicates(subset=['id', 'skill'], keep='last')

data_naruki['skill'] = data_naruki['skill'].apply(skill_transform)
data_naruki_final = data_naruki.groupby('id')['skill'].apply(list)
print len(data_naruki_final)

for skills in data_naruki_final:
    if len(skills) > 2 and skills not in data_train_w2v:
        musthave_dice_naruki_data.append(skills)
        
len(musthave_dice_naruki_data)

120107


119072

In [383]:
print ("Training model...")
musthave_dice_naruki_model = word2vec.Word2Vec(musthave_dice_naruki_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
          size=300, 
          min_count=3, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
          iter=4,
          sg =1
    )

musthave_dice_naruki_model.init_sims(replace=True)
test_case(musthave_dice_naruki_model, "musthave_dice_naruki_model")

2017-04-17 13:05:13,394 : INFO : collecting all words and their counts
2017-04-17 13:05:13,395 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-04-17 13:05:13,514 : INFO : PROGRESS: at sentence #10000, processed 217169 words, keeping 11788 word types
2017-04-17 13:05:13,572 : INFO : PROGRESS: at sentence #20000, processed 318186 words, keeping 21928 word types


Training model...


2017-04-17 13:05:13,629 : INFO : PROGRESS: at sentence #30000, processed 405501 words, keeping 30692 word types
2017-04-17 13:05:13,693 : INFO : PROGRESS: at sentence #40000, processed 493703 words, keeping 38555 word types
2017-04-17 13:05:13,750 : INFO : PROGRESS: at sentence #50000, processed 582980 words, keeping 45965 word types
2017-04-17 13:05:13,800 : INFO : PROGRESS: at sentence #60000, processed 674010 words, keeping 52084 word types
2017-04-17 13:05:13,851 : INFO : PROGRESS: at sentence #70000, processed 761402 words, keeping 58506 word types
2017-04-17 13:05:13,910 : INFO : PROGRESS: at sentence #80000, processed 847322 words, keeping 64909 word types
2017-04-17 13:05:13,961 : INFO : PROGRESS: at sentence #90000, processed 932263 words, keeping 70946 word types
2017-04-17 13:05:14,009 : INFO : PROGRESS: at sentence #100000, processed 1022074 words, keeping 75246 word types
2017-04-17 13:05:14,052 : INFO : PROGRESS: at sentence #110000, processed 1111899 words, keeping 80104

In [365]:
TEST_CASE

{'musthave_dice_model': [[(u'doctoral', 0.9114964604377747),
   (u'postdoc', 0.911190390586853),
   (u'hdfs', 0.9058669805526733),
   ('bigdata', 0.9043334722518921),
   (u'doctor', 0.8989962339401245),
   (u'dsp', 0.893810510635376),
   (u'hadoop', 0.8844509720802307),
   (u'data_science', 0.8758455514907837),
   (u'statistics', 0.8751969933509827),
   (u'recommendation', 0.8750705122947693)],
  [(u'css', 0.9601218700408936),
   (u'css3', 0.9592987298965454),
   (u'react', 0.9526255130767822),
   (u'prototype', 0.9475393295288086),
   (u'html5/css3/javascript', 0.9416552782058716),
   (u'css5', 0.9397596120834351),
   (u'framework', 0.9380757808685303),
   (u'cascade_style_sheet', 0.9371317625045776),
   (u'user_journey', 0.9369064569473267),
   (u'or_ruby_on_rail', 0.9366532564163208)],
  [(u'css', 0.9601218700408936),
   (u'css3', 0.9592987298965454),
   (u'react', 0.9526255130767822),
   (u'prototype', 0.9475393295288086),
   (u'html5/css3/javascript', 0.9416552782058716),
   (u'cs

In [366]:
TEST_CASE_TABLE = [ pd.DataFrame(TEST_CASE[t]) for t in TEST_CASE ]

In [367]:
[ t for t in TEST_CASE ]

['musthave_dice_model', 'musthave_model', 'musthave_dice_naruki_model']

In [368]:
TEST_CASE_TABLE[0]

,0,1,2,3,4,5,6,7,8,9
0,"(doctoral, 0.911496460438)","(postdoc, 0.911190390587)","(hdfs, 0.905866980553)","(bigdata, 0.904333472252)","(doctor, 0.89899623394)","(dsp, 0.893810510635)","(hadoop, 0.88445097208)","(data_science, 0.875845551491)","(statistics, 0.875196993351)","(recommendation, 0.875070512295)"
1,"(css, 0.960121870041)","(css3, 0.959298729897)","(react, 0.952625513077)","(prototype, 0.947539329529)","(html5/css3/javascript, 0.941655278206)","(css5, 0.939759612083)","(framework, 0.938075780869)","(cascade_style_sheet, 0.937131762505)","(user_journey, 0.936906456947)","(or_ruby_on_rail, 0.936653256416)"
2,"(css, 0.960121870041)","(css3, 0.959298729897)","(react, 0.952625513077)","(prototype, 0.947539329529)","(html5/css3/javascript, 0.941655278206)","(css5, 0.939759612083)","(framework, 0.938075780869)","(cascade_style_sheet, 0.937131762505)","(user_journey, 0.936906456947)","(or_ruby_on_rail, 0.936653256416)"
3,"(perl, 0.933317184448)","(aws, 0.932370722294)","(tool, 0.923393368721)","(administrator, 0.918892621994)","(map_reduce, 0.917342245579)","(ee, 0.911861658096)","(kinesis, 0.909421145916)","(elastic_beanstalk, 0.907735049725)","(ruby, 0.906385660172)","(scalable, 0.905459880829)"
4,"(css, 0.971244931221)","(ued, 0.959258317947)","(uxpin, 0.958706021309)","(prototype, 0.95783495903)","(ui_designer, 0.957764387131)","(framer, 0.957465529442)","(invision, 0.951822400093)","(angular, 0.951787173748)","(xd, 0.951317429543)","(cq, 0.950634479523)"
5,"(interaction, 0.978740692139)","(ui_design, 0.972190320492)","(frontends, 0.971920967102)","(user_experience_designer, 0.971385657787)","(quartz_composer, 0.970252275467)","(html5/css3/javascript, 0.969959080219)","(web_ux_design, 0.969360351562)","(visual, 0.969336986542)","(front_end_developer, 0.966380894184)","(websites, 0.966154873371)"
6,"(prototype, 0.972412645817)","(html, 0.971244931221)","(css3, 0.968775510788)","(ued, 0.964689016342)","(uxpin, 0.964391231537)","(ui_designer, 0.963480710983)","(invision, 0.960956335068)","(xd, 0.960825681686)","(javascript, 0.960121870041)","(quartz_composer, 0.959280252457)"
7,"(npm, 0.978815615177)","(less, 0.978117585182)","(bower, 0.976382493973)","(lamp, 0.972007393837)","(sharepoint, 0.970762729645)","(cq, 0.969783484936)","(three, 0.96908390522)","(responsive, 0.968459606171)","(bs/ms_in_computer_science, 0.967768013477)","(template, 0.966562569141)"
8,"(bower, 0.979354023933)","(require, 0.976583778858)","(gulp, 0.975829839706)","(npm, 0.975320875645)","(cq, 0.974446177483)","(load, 0.974304795265)","(template, 0.972091972828)","(mustache, 0.971216857433)","(adobe_aem, 0.969833254814)","(prototype, 0.969389259815)"


In [369]:
TEST_CASE_TABLE[1]

,0,1,2,3,4,5,6,7,8,9
0,"(data_mine, 0.969023704529)","(anomaly_detection, 0.934193253517)","(log_mine, 0.932994544506)","(parallel_program, 0.92199742794)","(algorithms, 0.919202327728)","(statistics, 0.912889242172)","(mapreduce, 0.907306015491)","(large_scale, 0.902518153191)","(odesign, 0.896661520004)","(math, 0.88308686018)"
1,"(jquery, 0.972502589226)","(less, 0.964989066124)","(bootstrap, 0.964809775352)","(sass, 0.963556289673)","(asp, 0.960511386395)","(ajax, 0.958946228027)","(jasmine, 0.957700073719)","(air_national_guard, 0.955925643444)","(dojo, 0.954901337624)","(drupal, 0.95435667038)"
2,"(jquery, 0.972502589226)","(less, 0.964989066124)","(bootstrap, 0.964809775352)","(sass, 0.963556289673)","(asp, 0.960511386395)","(ajax, 0.958946228027)","(jasmine, 0.957700073719)","(air_national_guard, 0.955925643444)","(dojo, 0.954901337624)","(drupal, 0.95435667038)"
3,"(engineer_in_test, 0.941640377045)","(ruby_on_rail, 0.936653494835)","(ror, 0.935851335526)","(debug, 0.930529117584)","(natural_language_process, 0.927911877632)","(perl, 0.925526976585)","(test_frameworks, 0.925350308418)","(jira, 0.92515963316)","(blackbox_test, 0.924978852272)","(test_automation, 0.924006104469)"
4,"(cq, 0.976207017899)","(css3, 0.972034454346)","(template, 0.972004175186)","(adobe_experience_management_suite, 0.97084939...","(javacript, 0.970072507858)","(adobe_cq, 0.967292129993)","(css, 0.966974258423)","(bs/ms_in_computer_science, 0.966046214104)","(xhtml, 0.964945077896)","(wordpress, 0.963612437248)"
5,"(css3, 0.974786937237)","(less, 0.962282776833)","(durandal, 0.959334015846)","(jquery, 0.95858758688)","(javascript, 0.953844368458)","(bootstrap, 0.951309561729)","(knockout, 0.951266944408)","(jquerry, 0.946676373482)","(adobe_experience_management_suite, 0.94567525...","(angular, 0.945338010788)"
6,"(html, 0.966974258423)","(cq, 0.948652029037)","(css3, 0.948174834251)","(template, 0.944355487823)","(d3, 0.940350234509)","(ccs3, 0.94015109539)","(adobe_experience_manager, 0.939906954765)","(xhtml, 0.939768671989)","(adobe_experience_management_suite, 0.93940067...","(adobe_cq, 0.93905133009)"
7,"(html/css, 0.990841805935)","(spa, 0.989920258522)","(react, 0.989896655083)","(knockout, 0.988486766815)","(adobe_experience_management_suite, 0.98804372...","(foundation, 0.987579286098)","(npm, 0.987313389778)","(durandal, 0.987174212933)","(coffeescript, 0.985966026783)","(bower, 0.985776424408)"
8,"(or_ruby_on_rail, 0.98610663414)","(pixi, 0.985155463219)","(strut, 0.98125565052)","(lamp, 0.980836033821)","(zend, 0.980259537697)","(ember, 0.98015421629)","(cors, 0.978867173195)","(webgl, 0.977693617344)","(css2, 0.977579116821)","(svg, 0.977298736572)"


In [370]:
TEST_CASE_TABLE[2]

,0,1,2,3,4,5,6,7,8,9
0,"(data_scientist, 0.908977508545)","(artificial_intelligence, 0.904796898365)","(algorithm_development, 0.897655010223)","(nlp, 0.890247046947)","(data_science, 0.886777579784)","(neural_network, 0.878467977047)","(random_forest, 0.873499691486)","(mllib, 0.869368672371)","(anomaly_detection, 0.869109392166)","(hivesql, 0.868712604046)"
1,"(ajax, 0.885636568069)","(css3, 0.867818593979)","(html5, 0.867108106613)","(web_development, 0.853417992592)","(jquery, 0.843765497208)","(ui_development, 0.837885260582)","(json, 0.82791030407)","(xhtml, 0.822524547577)","(bootstrap, 0.821585953236)","(css, 0.820676326752)"
2,"(ajax, 0.885636568069)","(css3, 0.867818593979)","(html5, 0.867108106613)","(web_development, 0.853417992592)","(jquery, 0.843765497208)","(ui_development, 0.837885260582)","(json, 0.82791030407)","(xhtml, 0.822524547577)","(bootstrap, 0.821585953236)","(css, 0.820676326752)"
3,"(ruby, 0.822397470474)","(perl, 0.785459041595)","(postgresql, 0.776874899864)","(bash, 0.766994595528)","(shell, 0.750223636627)","(open_source, 0.748502969742)","(site_reliability, 0.745224654675)","(relational_databases, 0.744770824909)","(shell_script, 0.742505669594)","(cloudcomputing, 0.742134273052)"
4,"(xhtml, 0.831475138664)","(ajax, 0.824178934097)","(web_development, 0.823227882385)","(javascript, 0.81837785244)","(html5, 0.816801190376)","(css3, 0.806224226952)","(ui_development, 0.803821206093)","(web_design, 0.793960213661)","(css, 0.790407538414)","(dhtml, 0.790007352829)"
5,"(css3, 0.948575258255)","(ui_development, 0.932035386562)","(ui_developer, 0.927372694016)","(bootstrap, 0.925883233547)","(media_query, 0.908752441406)","(object_orient_javascript, 0.905550122261)","(css2, 0.905142962933)","(css/javascript, 0.904003381729)","(object__orient_javascript, 0.903731822968)","(jqueryui, 0.903376638889)"
6,"(html5, 0.896988630295)","(ui_development, 0.860652089119)","(bootstrap, 0.853831768036)","(css3, 0.848497450352)","(web_development, 0.836186408997)","(ui_developer, 0.835832595825)","(web_tool, 0.827304422855)","(xhtml, 0.825168967247)","(web_program, 0.821044266224)","(javascript, 0.820676326752)"
7,"(node, 0.941638767719)","(bootstrap, 0.923335909843)","(react, 0.916856706142)","(backbone, 0.897712349892)","(knockout, 0.87665104866)","(html5, 0.870873451233)","(ui_developer, 0.862407147884)","(ui_development, 0.856093883514)","(frontend, 0.855454802513)","(bower, 0.854888439178)"
8,"(react, 0.958459377289)","(angular, 0.941638708115)","(backbone, 0.939422428608)","(mongodb, 0.91990685463)","(mean_stack, 0.917893469334)","(bower, 0.915401279926)","(mean, 0.914160728455)","(full_stack_developer, 0.912163376808)","(bootstrap, 0.909401893616)","(ui_architect, 0.909393429756)"


In [371]:
musthave_dice_naruki_model.wv.save_word2vec_format("musthave_dice_naruki.model")

2017-04-17 13:04:28,660 : INFO : storing 84813x300 projection weights into musthave_dice_naruki.model


In [372]:
!ls -l

total 311200
-rw-r--r-- 1 root root  17781151 12:57 17 Th04 musthave_dice_naruki_100d.model
-rw-r--r-- 1 root root 242934950 13:04 17 Th04 musthave_dice_naruki.model
-rw-r--r-- 1 root root   8885546 15:44 16 Th04 mustHaveSkills-2.csv
-rw-r--r-- 1 root root  18240737 11:17 17 Th04 naukri_skill_full
-rw-r--r-- 1 root root   1276232 15:57 16 Th04 prepare_data.ipynb
-rw-r--r-- 1 root root   3972850 15:54 16 Th04 prep_data_tokens_underscore_1
-rw-r--r-- 1 root root     34398 13:04 17 Th04 skill2vec_ver2.ipynb
-rw-r--r-- 1 root root     77333 15:55 16 Th04 word2vec_skills.ipynb
-rw-r--r-- 1 root root     12329 16:33 16 Th04 word2vec_skills_nltk.ipynb


In [396]:
musthave_dice_naruki_model_100d = word2vec.Word2Vec(musthave_dice_naruki_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
          size=100, 
          min_count=5, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
          iter=4,
          sg =1
    )

musthave_dice_naruki_model_200d = word2vec.Word2Vec(musthave_dice_naruki_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
          size=200, 
          min_count=5, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
          iter=4,
          sg =1
    )

2017-04-17 13:38:09,846 : INFO : collecting all words and their counts
2017-04-17 13:38:09,847 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-04-17 13:38:10,020 : INFO : PROGRESS: at sentence #10000, processed 217169 words, keeping 11788 word types
2017-04-17 13:38:10,084 : INFO : PROGRESS: at sentence #20000, processed 318186 words, keeping 21928 word types
2017-04-17 13:38:10,148 : INFO : PROGRESS: at sentence #30000, processed 405501 words, keeping 30692 word types
2017-04-17 13:38:10,201 : INFO : PROGRESS: at sentence #40000, processed 493703 words, keeping 38555 word types
2017-04-17 13:38:10,252 : INFO : PROGRESS: at sentence #50000, processed 582980 words, keeping 45965 word types
2017-04-17 13:38:10,307 : INFO : PROGRESS: at sentence #60000, processed 674010 words, keeping 52084 word types
2017-04-17 13:38:10,360 : INFO : PROGRESS: at sentence #70000, processed 761402 words, keeping 58506 word types
2017-04-17 13:38:10,414 : INFO : PROGRESS: at 

In [397]:
musthave_dice_naruki_model_100d.wv.save_word2vec_format("musthave_dice_naruki_100d.model")
musthave_dice_naruki_model_200d.wv.save_word2vec_format("musthave_dice_naruki_200d.model")

2017-04-17 13:40:14,610 : INFO : storing 18426x100 projection weights into musthave_dice_naruki_100d.model
2017-04-17 13:40:16,028 : INFO : storing 18426x200 projection weights into musthave_dice_naruki_200d.model


In [400]:
!ls -l
!free -m

total 83640
-rw-r--r-- 1 root root     1069 13:19 17 Th04 LICENSE
-rw-r--r-- 1 root root 17771251 13:40 17 Th04 musthave_dice_naruki_100d.model
-rw-r--r-- 1 root root 35249875 13:40 17 Th04 musthave_dice_naruki_200d.model
-rw-r--r-- 1 root root  8885546 15:44 16 Th04 mustHaveSkills-2.csv
-rw-r--r-- 1 root root 18240737 11:17 17 Th04 naukri_skill_full
-rw-r--r-- 1 root root  1276232 15:57 16 Th04 prepare_data.ipynb
-rw-r--r-- 1 root root  3972850 15:54 16 Th04 prep_data_tokens_underscore_1
-rw-r--r-- 1 root root       42 13:19 17 Th04 README.md
-rw-r--r-- 1 root root   138294 13:38 17 Th04 skill2vec_ver2.ipynb
drwxr-xr-x 2 root root       46 13:19 17 Th04 test
drwxr-xr-x 2 root root       63 13:19 17 Th04 word2vec_skills
-rw-r--r-- 1 root root    77333 15:55 16 Th04 word2vec_skills.ipynb
-rw-r--r-- 1 root root    12329 16:33 16 Th04 word2vec_skills_nltk.ipynb
              total        used        free      shared  buff/cache   available
Mem:         128673       16449       42482      

In [384]:
musthave_dice_naruki_model_100d.similar_by_word(skill_transform('machine learning'))

[(u'data_scientist', 0.9083810448646545),
 ('nlp', 0.8880348205566406),
 (u'algorithm_development', 0.8754720091819763),
 (u'recommendation', 0.8676395416259766),
 (u'random_forest', 0.861264705657959),
 (u'anomaly_detection', 0.8607847690582275),
 (u'data_science', 0.8525664806365967),
 (u'caffe', 0.8518109321594238),
 (u'neural_network', 0.8495544791221619),
 (u'recommender', 0.8487647771835327)]

In [385]:
musthave_dice_naruki_model.similar_by_word(skill_transform('machine learning'))

[(u'data_scientist', 0.8998317122459412),
 (u'algorithm_development', 0.8750029802322388),
 ('nlp', 0.8717856407165527),
 (u'artificial_intelligence', 0.8528630137443542),
 (u'data_science', 0.8504092693328857),
 (u'anomaly_detection', 0.8483552932739258),
 (u'hivesql', 0.8452751636505127),
 (u'tensorflow', 0.8447992205619812),
 (u'random_forest', 0.842191219329834),
 (u'recommendation', 0.8411462306976318)]

In [391]:
musthave_dice_naruki_model.similar_by_word(skill_transform('ux'))

[(u'user_experience_design', 0.8861395716667175),
 (u'visual_design', 0.8819597959518433),
 (u'ux_design', 0.8733750581741333),
 (u'user_interface_design', 0.8626808524131775),
 (u'interaction_design', 0.8534577488899231),
 (u'usability', 0.8527886867523193),
 (u'interaction_designer', 0.8406970500946045),
 (u'wire_frame', 0.8393874168395996),
 (u'ux_designer', 0.83745276927948),
 (u'user_experience/ux', 0.8368674516677856)]

In [379]:
musthave_dice_naruki_model.similar_by_word(skill_transform('javascript'))

[(u'ajax', 0.885636568069458),
 (u'css3', 0.8678185939788818),
 (u'html5', 0.8671081066131592),
 (u'web_development', 0.8534179925918579),
 (u'jquery', 0.8437654972076416),
 (u'ui_development', 0.8378852605819702),
 (u'json', 0.827910304069519),
 (u'xhtml', 0.8225245475769043),
 (u'bootstrap', 0.8215859532356262),
 (u'css', 0.820676326751709)]

In [395]:
musthave_dice_naruki_model.similar_by_word(skill_transform('node_js'), 300)

[(u'react', 0.9416168332099915),
 (u'backbone', 0.9306129813194275),
 (u'angular', 0.9224972128868103),
 (u'mean_stack', 0.9146184921264648),
 (u'bower', 0.9023303985595703),
 (u'mean', 0.8987296223640442),
 (u'mongodb', 0.8956993222236633),
 (u'grunt', 0.8945599794387817),
 (u'npm', 0.8914229869842529),
 (u'full_stack_developer', 0.8905653953552246),
 (u'mongo_db', 0.8900968432426453),
 (u'css/javascript', 0.8900823593139648),
 (u'bootstrap', 0.8898901343345642),
 (u'mocha', 0.8849277496337891),
 (u'javascript_developer', 0.8823011517524719),
 (u'ui_architect', 0.8822310566902161),
 (u'express', 0.8795454502105713),
 (u'gulp', 0.8791297674179077),
 (u'responsive_web', 0.8787859678268433),
 (u'electronic_city', 0.8785770535469055),
 (u'front_end_web_developer', 0.8777059316635132),
 (u'less', 0.8755059242248535),
 (u'frontend', 0.8755017518997192),
 (u'extension', 0.8748639225959778),
 (u'mean_stack_developer', 0.8732047080993652),
 (u'loopback', 0.8720988631248474),
 (u'ruby_rail', 0.

In [381]:
musthave_dice_naruki_model_100d.similar_by_word(skill_transform('node_js'), 300)

[(u'react', 0.9322503805160522),
 (u'backbone', 0.9266782999038696),
 (u'mean_stack', 0.9251165986061096),
 (u'angular', 0.9170751571655273),
 (u'mean', 0.9123091697692871),
 (u'grunt', 0.9012008905410767),
 (u'mean_stack_developer', 0.8988714218139648),
 (u'express', 0.89606773853302),
 (u'bower', 0.8914059996604919),
 (u'npm', 0.8903093338012695),
 (u'loopback', 0.8901658058166504),
 (u'ui_architect', 0.890097439289093),
 (u'gulp', 0.88966965675354),
 (u'mongodb', 0.8894267082214355),
 (u'full_stack_developer', 0.8890975713729858),
 (u'extension', 0.8877204656600952),
 (u'javascript_developer', 0.8874136209487915),
 (u'bootstrap', 0.8830759525299072),
 (u'meanstack', 0.8820235729217529),
 (u'ruby_rail', 0.8796218633651733),
 (u'mongo_db', 0.8789615631103516),
 (u'front__end_developer', 0.878875195980072),
 (u'backend_developer', 0.8750494122505188),
 (u'html5_and_css3', 0.8740687370300293),
 (u'mocha', 0.8730786442756653),
 (u'api_design', 0.8726017475128174),
 (u'firebase', 0.872273

In [382]:
re.sub(r'^angular.*$', 'angular', "angular.js")

'angular'